<a href="https://colab.research.google.com/github/Minhvt34/Forced_Alignment_Pytorch/blob/main/vietTTS_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Quick start

In [1]:
%cd /content
!git clone https://github.com/NTT123/vietTTS.git
%cd /content/vietTTS
!pip3 install -e .
!bash ./scripts/quick_start.sh
from google.colab import output
output.clear()

In [4]:
text = """
Trăng cô đơn lặng lẽ Nhớ ai qua từng. Góc phố ngày xưa. Nghe trong tim vụn vỡ. Bao lần nát tan vì ai. Có ai nhớ ngày Xưa em ơi Giờ đã xa tôi mất rồi Ánh trăng sáng Lòng anh em ơi Giờ thấm ướt
"""
!python3 -m vietTTS.synthesizer --lexicon-file assets/infore/lexicon.txt --text="{text}" --output=clip.wav --silence-duration 0.2
import IPython.display as ipd
ipd.Audio('clip.wav')

Normalized text input: trăng cô đơn lặng lẽ nhớ ai qua từng sil góc phố ngày xưa sil nghe trong tim vụn vỡ sil bao lần nát tan vì ai sil có ai nhớ ngày xưa em ơi giờ đã xa tôi mất rồi ánh trăng sáng lòng anh em ơi giờ thấm ướt
writing output to file clip.wav


# Train duration model and acoustic model

In [ ]:
!bash ./scripts/download_aligned_infore_dataset.sh

/content/vietTTS /content/vietTTS
Downloading...
From: https://drive.google.com/uc?id=1Pe-5lKT_lZsliv2WxQDai2mjhI9ZMFlj
To: /content/vietTTS/train_data/infore.zip
100% 2.44G/2.44G [00:12<00:00, 189MB/s]
/content/vietTTS


In [ ]:
!python3 -m vietTTS.nat.duration_trainer

In [ ]:
!python3 -m vietTTS.nat.acoustic_trainer

---------------------------  -------------------------
duration_lstm_dim            256
vocab_size                   256
duration_embed_dropout_rate  0.5
num_training_steps           200000
postnet_dim                  512
acoustic_decoder_dim         512
acoustic_encoder_dim         256
max_phoneme_seq_len          384
max_wave_len                 196608
special_phonemes             ['sil', 'sp', 'spn', ' ']
sil_index                    0
sp_index                     1
word_end_index               3
mel_dim                      80
n_fft                        1024
sample_rate                  16000
fmin                         0.0
fmax                         8000
batch_size                   64
learning_rate                0.0001
duration_learning_rate       0.0001
max_grad_norm                1.0
weight_decay                 0.0001
ckpt_dir                     assets/infore/nat
data_dir                     train_data
---------------------------  -------------------------
Resuming fr

# Train HiFiGAN vocoder

In [ ]:
%cd /content/vietTTS
!bash ./scripts/download_aligned_infore_dataset.sh


/content/vietTTS
/content/vietTTS /content/vietTTS
Downloading...
From: https://drive.google.com/uc?id=1Pe-5lKT_lZsliv2WxQDai2mjhI9ZMFlj
To: /content/vietTTS/train_data/infore.zip
2.44GB [00:21, 115MB/s]
replace 00945.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!git clone https://github.com/jik876/hifi-gan.git
%cd hifi-gan
!ln -sf /content/vietTTS/train_data /content/vietTTS/hifi-gan/data
!cd data;  ls -1 *.TextGrid| sed -e 's/\.TextGrid$//' > /content/vietTTS/hifi-gan/files.txt
!head -n 100 ./files.txt > val_files.txt
!tail -n +101 files.txt > train_files.txt
!cp /content/vietTTS/assets/hifigan/config.json ./config.json

In [ ]:
!python3 train.py --config config.json --input_wavs_dir=data  --input_training_file=train_files.txt  --input_validation_file=val_files.txt